<a href="https://colab.research.google.com/github/zemersonnn/Home_Sales/blob/main/Home_Sales_Fixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz

In [2]:
import os

In [3]:
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

In [4]:
!pip install findspark

In [5]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=66d7f72a547342efcf5c18622f0a5195821e8b3bbba642318c01f63d6bdb92fe
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [7]:
! pip install py4j

In [8]:
! pip uninstall py4j

Found existing installation: py4j 0.10.9.7
Uninstalling py4j-0.10.9.7:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/py4j-0.10.9.7.dist-info/*
    /usr/local/lib/python3.10/dist-packages/py4j/*
    /usr/local/share/py4j/py4j0.10.9.7.jar
Proceed (Y/n)? y
  Successfully uninstalled py4j-0.10.9.7


In [10]:
! pip install py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 1.7 MB/s eta 0:00:00


In [13]:
import os

# Get the current user's home directory
home_directory = os.path.expanduser("~")

# Set the SPARK_HOME environment variable
os.environ["SPARK_HOME"] = f"{home_directory}/spark-3.0.1-bin-hadoop3.2"

In [14]:
# Print the value of the SPARK_HOME environment variable
print(os.environ["SPARK_HOME"])

/root/spark-3.0.1-bin-hadoop3.2


In [15]:
# Restart the Python interpreter to ensure the environment variable is updated
exit()

In [1]:
# Import and initialize findspark
import findspark
findspark.init()

# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Testing").getOrCreate()

# Create a Spark DataFrame
df = spark.createDataFrame([
(0, "First row"),
(1, "Second row"),
(2, "Third row")
], ["ids", "rows"])

df.show()

+---+----------+
|ids|      rows|
+---+----------+
|  0| First row|
|  1|Second row|
|  2| Third row|
+---+----------+



In [2]:
# Import findspark and initialize.
import findspark
findspark.init()

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [ ]:
# 2. Create a temporary view of the DataFrame.